In [ ]:
# Copyright 2022-2023 Sony Semiconductor Solutions Corp. All rights reserved.
#
# Licensed under the Apache License, Version 2.0 (the "License");
# you may not use this file except in compliance with the License.
# You may obtain a copy of the License at
#
#     http://www.apache.org/licenses/LICENSE-2.0
#
# Unless required by applicable law or agreed to in writing, software
# distributed under the License is distributed on an "AS IS" BASIS,
# WITHOUT WARRANTIES OR CONDITIONS OF ANY KIND, either express or implied.
# See the License for the specific language governing permissions and
# limitations under the License.


# Deploy "**Edge Application**" to Device

This notebook explains the workflow to deploy "**Edge Applications**" from Edge AI Device to "**Console for AITRIOS**" using "**Console Access Library**".

Instructions are described in [README.md](./README.md).

## Imports

In [ ]:
import errno
import json
from pathlib import Path

import jsonschema
import pandas as pd
from console_access_library.client import Client
from IPython.display import display


## Load Configurations

Load the configuration file and set the variables.

In [ ]:
def validate_symlink(path: Path):
    if path.is_symlink():
        raise OSError(
            errno.ELOOP,
            "Symbolic link is not supported. Please use real folder or file",
            f"{path}",
        )


configuration_path = Path("./configuration.json")
validate_symlink(configuration_path)

# Load configuration file.
with open(configuration_path, "r") as f:
    json_load = json.load(f)

configuration_schema_path = Path("./configuration_schema.json")
validate_symlink(configuration_schema_path)

# Load configuration schema file.
with open(configuration_schema_path, "r") as f:
    json_schema = json.load(f)

# Validate configuration.
jsonschema.validate(json_load, json_schema)


## Load an instance of "**Console Access Library**" Client.
Run this cell to restore instance variable 'client_obj' created on a notebook of Set up "**Console Access Library**".

In [ ]:
%store -r
client_obj: Client = client_obj

## Deploy "**Edge Application**" to edge AI device

In [ ]:
# Get an instance of Deployment API.
deployment_obj = client_obj.get_deployment()

param_dict = json_load.copy()

device_ids = param_dict["device_ids"]
device_ids_join = ",".join(device_ids)
param_dict["device_ids"] = device_ids_join


# Call an API to deploy Vision and Sensing Application from Console for AITRIOS to Edge AI Device.
try:
    response = deployment_obj.deploy_device_app(**param_dict)
except Exception as e:
    # EXCEPTION
    raise e

# Response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS
print(
    "Start to deploy application. \n\tapp_name: "
    + json_load["app_name"]
    + "\n\tversion_number: "
    + json_load["version_number"]
    + "\n\tdevice_ids: "
    + device_ids_join
)


## Check deployment status
Deploying to devices takes time. <br>
To complete the deployment, ensure that the deployment status is **`Success`**. <br>
After you start the deployment, run the following code cell to check the status :

In [ ]:
status_dictionary = {"0": "Deploying", "1": "Success", "2": "Fail", "3": "Cancel"}

response_statuses = []
response_device_ids = []

# Call an API to get Vision and Sensing Application info.
try:
    response = deployment_obj.get_device_app_deploys(
        json_load["app_name"], json_load["version_number"]
    )
except Exception as e:
    # EXCEPTION
    raise e

# response error check
if "result" in response and response["result"] != "SUCCESS":
    # ERROR
    raise ValueError("ERROR", response)

# SUCCESS

deploys = response.get("deploys", [])
# Display deployment status of specified devices
for deploy in deploys:
    if "devices" in deploy:
        devices = deploy.get("devices", [])
        for device in devices:
            if (
                "device_id" in device
                and device["device_id"] in device_ids
                and device["latest_deployment_flg"] == "1"
            ):
                response_device_ids.append(device.get("device_id", ""))
                deploy_status = device.get("status", "")
                response_statuses.append(
                    status_dictionary.get(
                        deploy_status, "Unknown status '" + deploy_status + "'"
                    )
                )

if len(response_device_ids) == 0:
    raise Exception(
        "Failed to get deploy status. Deploy history not found. \n\tapp_name: "
        + json_load["app_name"]
        + "\n\tversion_number: "
        + json_load["version_number"]
        + "\n\tdevice_ids: "
        + device_ids_join
    )

print(
    "Deployment status of: \n\tapp_name: "
    + json_load["app_name"]
    + "\n\tversion_number: "
    + json_load["version_number"]
)

output_frame = pd.DataFrame(
    {"device_id": response_device_ids, "status": response_statuses}
)

# setting backup
backup_max_rows = pd.options.display.max_rows
# output limit clear
pd.set_option("display.max_rows", None)
# View results
display(output_frame)
# setting restore
pd.set_option("display.max_rows", backup_max_rows)
